In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import os
import numpy as np
import math
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from keras.layers import Masking

Using TensorFlow backend.
C:\Users\silva\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\silva\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\silva\AppData\Roaming\Python\Python36\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\silva\AppData\Roa

In [2]:
days = 10

In [3]:
def scale(training_set):
    # Feature Scaling
    sc = MinMaxScaler(feature_range = (0, 1))
    training_set_scaled = sc.fit_transform(training_set)
    # Creating a data structure with 'days' time-steps and 1 output
    X_train = []
    y_train = []
    for i in range(days, training_set.shape[0]):
        X_train.append(training_set_scaled[i-days:i, 0])
        y_train.append(training_set_scaled[i, 0])
    X_train, y_train = np.array(X_train), np.array(y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    return X_train, y_train, sc

In [4]:
def train_LSTM(X_train, y_train):
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape = (X_train.shape[1], 1)))
    #Adding the first LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    model.add(Dropout(0.2))
    # Adding a second LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))
    # Adding a third LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50, return_sequences = True))
    model.add(Dropout(0.2))
    # Adding a fourth LSTM layer and some Dropout regularisation
    model.add(LSTM(units = 50))
    model.add(Dropout(0.2))
    # Adding the output layer
    model.add(Dense(units = 1))

    # Compiling the RNN
    model.compile(optimizer = 'adam', loss = 'mean_squared_error')

    # Fitting the RNN to the Training set
    model.fit(X_train, y_train, epochs = 30, batch_size = 32)
    return model

In [5]:
def predict(dataset_train, dataset_test, model, sc):
    dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)
    inputs = dataset_total[len(dataset_total) - dataset_test.shape[0] - days:].values
    inputs = inputs.reshape(-1,1)
    inputs = sc.transform(inputs)
    X_test = []
    for i in range(days, inputs.shape[0]):
        X_test.append(inputs[i-days:i, 0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    print(X_test.shape)
    # (273, 'days', 1)
    predicted_stock_price = model.predict(X_test)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)
    return X_test, predicted_stock_price

In [6]:
def plot_results(dataset_test, predicted_stock_price):
    # Visualising the results
    plt.plot(dataset_test.values, color = 'red', label = 'Real')
    plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted')
    plt.xticks(np.arange(0,dataset_test.shape[0],50))
    plt.xlabel('Time')
    plt.ylabel('Index')
    plt.legend()
    plt.show()

In [7]:
def impute_with_prediction(df_orig, df_imputed):
    for i in df_orig.index:
        if pd.isna(df_orig.loc[i]):
            df_orig.at[i] = df_imputed.loc[i]
    return df_orig

In [8]:
df = pd.read_csv(r'output\/CompleteCovid.csv',index_col=0)

In [ ]:
for i in range(0,30):
    df_train_complete = None
    df_test_complete = None
    for col in df.columns:       

        test_set = df.loc['2020-02-26 02:30:00':, col].fillna(-1).values #Selecting test data
        training_set = df.loc['2020-01-22 09:00:00':'2020-02-26 02:00:00', col].fillna(-1).values #Selecting training data
        training_set = training_set.reshape(-1, 1)
        X_train, y_train, sc = scale(training_set)
        model = train_LSTM(X_train, y_train)
        # Dados de treinamento
        predicted_stock_price_train = model.predict(X_train)
        predicted_stock_price_train = sc.inverse_transform(predicted_stock_price_train)
        #plot_results(df.loc['2000-01-01':'2015-01-01', col].fillna(-1), predicted_stock_price_train)
        # Dados de teste
        X_test, predicted_stock_price = predict(df.loc['2020-01-22 09:00:00':'2020-02-26 02:00:00', col].fillna(-1), df.loc['2020-02-26 02:30:00':, col].fillna(-1), model, sc)
        #plot_results(df.loc['2015-01-02':, col].fillna(-1), predicted_stock_price)

        vazio = np.empty(days)
        predicted_stock_price_train = np.append(vazio, predicted_stock_price_train)
        series_complete_train = impute_with_prediction(df.loc['2020-01-22 09:00:00':'2020-02-26 02:00:00', col], pd.DataFrame(index=df.loc['2020-01-22 09:00:00':'2020-02-26 02:00:00', col].index, data=predicted_stock_price_train))
        df_train_complete = pd.concat([df_train_complete, pd.DataFrame(index=df.loc['2020-01-22 09:00:00':'2020-02-26 02:00:00', col].index, data=series_complete_train, columns=[col])], axis=1, join='outer')
        df_test_complete = pd.concat([df_test_complete, pd.DataFrame(data=predicted_stock_price, index = df.loc['2020-02-26 02:30:00':, col].index, columns = [col])], axis=1, join='outer')
    df_train_complete.to_csv(r'output\/lstm\/CompleteCovidTrainLSTM10j_'+str(i)+'.csv', index = True)
    df_test_complete.to_csv(r'output\/lstm\/CompleteCovidTestLSTM10j_'+str(i)+'.csv', index = True)

Epoch 1/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0058
Epoch 2/30
1657/1657 [==============================] - 2s 955us/step - loss: 0.0058
Epoch 3/30
1657/1657 [==============================] - 2s 980us/step - loss: 0.0058
Epoch 4/30
1657/1657 [==============================] - 2s 974us/step - loss: 0.0058
Epoch 5/30
1657/1657 [==============================] - 2s 961us/step - loss: 0.0058 0s - lo
Epoch 6/30
1657/1657 [==============================] - 2s 979us/step - loss: 0.0058 0s - 
Epoch 7/30
1657/1657 [==============================] - 2s 970us/step - loss: 0.0058
Epoch 8/30
1657/1657 [==============================] - 2s 975us/step - loss: 0.0058
Epoch 9/30
1657/1657 [==============================] - 2s 973us/step - loss: 0.0058
Epoch 10/30
1657/1657 [==============================] - 2s 973us/step - loss: 0.0058
Epoch 11/30
1657/1657 [==============================] - 2s 962us/step - loss: 0.0058
Epoch 12/30
1657/1657 [============================

1657/1657 [==============================] - 2s 1ms/step - loss: 0.0010
Epoch 13/30
1657/1657 [==============================] - 2s 1ms/step - loss: 0.0010
Epoch 14/30
1657/1657 [==============================] - 2s 1ms/step - loss: 0.0010
Epoch 15/30
1657/1657 [==============================] - 2s 1ms/step - loss: 0.0010
Epoch 16/30
1657/1657 [==============================] - 2s 1ms/step - loss: 0.0010
Epoch 17/30
1657/1657 [==============================] - 2s 1ms/step - loss: 0.0010
Epoch 18/30
1657/1657 [==============================] - 2s 1ms/step - loss: 0.0010
Epoch 19/30
1657/1657 [==============================] - 2s 1ms/step - loss: 0.0010
Epoch 20/30
1657/1657 [==============================] - 2s 1ms/step - loss: 0.0010
Epoch 21/30
1657/1657 [==============================] - 2s 1ms/step - loss: 0.0010
Epoch 22/30
1657/1657 [==============================] - 2s 1ms/step - loss: 0.0010
Epoch 23/30
1657/1657 [==============================] - 2s 1ms/step - loss: 0.0010- ET


1657/1657 [==============================] - 2s 1ms/step - loss: 6.1123e-04
Epoch 25/30
1657/1657 [==============================] - 2s 1ms/step - loss: 6.0665e-04
Epoch 26/30
1657/1657 [==============================] - 2s 1ms/step - loss: 6.0275e-04
Epoch 27/30
1657/1657 [==============================] - 2s 1ms/step - loss: 6.1280e-04
Epoch 28/30
1657/1657 [==============================] - 2s 1ms/step - loss: 6.1015e-04
Epoch 29/30
1657/1657 [==============================] - 2s 1ms/step - loss: 6.0955e-04
Epoch 30/30
1657/1657 [==============================] - 2s 1ms/step - loss: 6.0540e-04
(715, 10, 1)
Epoch 1/30
1657/1657 [==============================] - 13s 8ms/step - loss: 6.1351e-04
Epoch 2/30
1657/1657 [==============================] - 2s 1ms/step - loss: 6.1516e-04
Epoch 3/30
1657/1657 [==============================] - 2s 1ms/step - loss: 6.1146e-04
Epoch 4/30
1657/1657 [==============================] - 2s 1ms/step - loss: 6.1346e-04
Epoch 5/30
1657/1657 [============

1657/1657 [==============================] - 2s 1ms/step - loss: 0.0013
Epoch 4/30
1657/1657 [==============================] - 2s 1ms/step - loss: 0.0013
Epoch 5/30
1657/1657 [==============================] - 2s 1ms/step - loss: 0.0012A: 1s - l
Epoch 6/30
1657/1657 [==============================] - 2s 1ms/step - loss: 0.0012
Epoch 7/30
1657/1657 [==============================] - 2s 1ms/step - loss: 0.0013
Epoch 8/30
1657/1657 [==============================] - 2s 1ms/step - loss: 0.0013
Epoch 9/30
1657/1657 [==============================] - 2s 1ms/step - loss: 0.0013
Epoch 10/30
1657/1657 [==============================] - 2s 1ms/step - loss: 0.0012
Epoch 11/30
1657/1657 [==============================] - 3s 2ms/step - loss: 0.0013
Epoch 12/30
1657/1657 [==============================] - 4s 3ms/step - loss: 0.0012
Epoch 13/30
1657/1657 [==============================] - 4s 2ms/step - loss: 0.0013
Epoch 14/30
1657/1657 [==============================] - 4s 3ms/step - loss: 0.0013
E

1657/1657 [==============================] - 3s 2ms/step - loss: 0.0101
Epoch 15/30
1657/1657 [==============================] - 3s 2ms/step - loss: 0.0101
Epoch 16/30
1657/1657 [==============================] - 3s 2ms/step - loss: 0.0101
Epoch 17/30
1657/1657 [==============================] - 3s 2ms/step - loss: 0.0101
Epoch 18/30
1657/1657 [==============================] - 3s 2ms/step - loss: 0.0101
Epoch 19/30
1657/1657 [==============================] - 3s 2ms/step - loss: 0.0101
Epoch 20/30
1657/1657 [==============================] - 3s 2ms/step - loss: 0.0101
Epoch 21/30
1657/1657 [==============================] - 3s 2ms/step - loss: 0.0101
Epoch 22/30
1657/1657 [==============================] - 3s 2ms/step - loss: 0.0101
Epoch 23/30
1657/1657 [==============================] - 3s 2ms/step - loss: 0.0101
Epoch 24/30
1657/1657 [==============================] - 3s 2ms/step - loss: 0.0101
Epoch 25/30
1657/1657 [==============================] - 3s 2ms/step - loss: 0.0101
Epoc

...

1657/1657 [==============================] - 7s 4ms/step - loss: 0.0011
Epoch 8/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0011
Epoch 9/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0011
Epoch 10/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0011
Epoch 11/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0011
Epoch 12/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0011
Epoch 13/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0011
Epoch 14/30
1657/1657 [==============================] - 7s 5ms/step - loss: 0.0011
Epoch 15/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0011- ETA: 2s - l
Epoch 16/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0011- ETA: 2s - 
Epoch 17/30
1657/1657 [==============================] - 6s 4ms/step - loss: 0.0011
Epoch 18/30
1657/1657 [==============================] - 6s 4ms/s

C:\Users\silva\Anaconda3\envs\mestrado\lib\site-packages\keras\callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.242133). Check your callbacks.
  % delta_t_median)


1657/1657 [==============================] - 375s 226ms/step - loss: 0.0023 12:3
Epoch 2/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0023
Epoch 3/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0023
Epoch 4/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0022
Epoch 5/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0022
Epoch 6/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0023
Epoch 7/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0022
Epoch 8/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0022
Epoch 9/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0023
Epoch 10/30
1657/1657 [==============================] - 8s 5ms/step - loss: 0.0022
Epoch 11/30
1657/1657 [==============================] - 8s 5ms/step - loss: 0.0023
Epoch 12/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0023
Epo

1657/1657 [==============================] - 10s 6ms/step - loss: 0.0101
Epoch 21/30
1657/1657 [==============================] - 8s 5ms/step - loss: 0.0101
Epoch 22/30
1657/1657 [==============================] - 7s 5ms/step - loss: 0.0101
Epoch 23/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0101
Epoch 24/30
1657/1657 [==============================] - 8s 5ms/step - loss: 0.0101
Epoch 25/30
1657/1657 [==============================] - 8s 5ms/step - loss: 0.0101
Epoch 26/30
1657/1657 [==============================] - 7s 4ms/step - loss: 0.0101
Epoch 27/30
1657/1657 [==============================] - 8s 5ms/step - loss: 0.0101
Epoch 28/30
1657/1657 [==============================] - 8s 5ms/step - loss: 0.0101
Epoch 29/30
1657/1657 [==============================] - 8s 5ms/step - loss: 0.0101
Epoch 30/30
1657/1657 [==============================] - 8s 5ms/step - loss: 0.0101A: 4s - lo - E
(715, 10, 1)
Epoch 1/30
1657/1657 [==============================] - 357s

1657/1657 [==============================] - 8s 5ms/step - loss: 5.9190e-04
Epoch 13/30
1657/1657 [==============================] - 8s 5ms/step - loss: 5.9395e-04A: 4s  - ETA: 1s - loss: 
Epoch 14/30
1657/1657 [==============================] - 8s 5ms/step - loss: 5.9355e-04
Epoch 15/30
1657/1657 [==============================] - 8s 5ms/step - loss: 5.9335e-04
Epoch 16/30
1657/1657 [==============================] - 8s 5ms/step - loss: 5.9035e-04
Epoch 17/30
1657/1657 [==============================] - 8s 5ms/step - loss: 5.9020e-04A: - ETA: 0s - loss: 6.477
Epoch 18/30
1657/1657 [==============================] - 7s 4ms/step - loss: 6.0020e-04A: 0s - loss: 6.75
Epoch 19/30
1657/1657 [==============================] - 8s 5ms/step - loss: 5.9135e-04
Epoch 20/30
1657/1657 [==============================] - 8s 5ms/step - loss: 5.9480e-04A: 2s - lo
Epoch 21/30
1657/1657 [==============================] - 8s 5ms/step - loss: 5.8848e-04
Epoch 22/30
1657/1657 [==============================